In [1]:
from pyspark import SparkConf , SparkContext

In [2]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("wordcount")
sc = SparkContext(conf = conf)
sc

<SparkContext master=local appName=wordcount>

In [3]:
def mapper1try(line):
        wordlist = line.split(" ")
        newitem = []
        maplist = []

        for item in wordlist:

                c = item.split(",")
   
                for unit in c:

                    if unit=='M':
                           pass
                    elif unit=='N':
                           pass
                    else:    
                            
                            unit = int(unit)
                            
                    newitem.append(unit)
               
                 
                maplist.append(((newitem[1],newitem[2],newitem[3])))
                
    
        
            
        return maplist

In [4]:
def mapper2try(line):
        wordlist = line.split(" ")
        newitem = []
        naplist = []

        for item in wordlist:

                c = item.split(",")
   
                for unit in c:

                    if unit=='M':
                           pass
                    elif unit=='N':
                           pass
                    else:    
                            
                            unit = int(unit)
                            
                    newitem.append(unit)
               
                 
                naplist.append(((newitem[2],newitem[1],newitem[3])))
                
    
        
            
        return naplist

In [6]:
lines1 = sc.textFile("/Users/huangyushu/Desktop/input.txt").flatMap(mapper1try)

lines1=lines1.map(lambda x:(x[1],x[2]))#只取jv
lines1 = lines1.take(250000)
lines1= sc.parallelize(lines1)

print(lines1.collect())

[(0, 10), (1, 0), (2, 20), (0, 0), (1, 30), (2, 0), (0, 40), (1, 0), (2, 50)]


In [7]:
lines2 = sc.textFile("/Users/huangyushu/Desktop/input.txt").flatMap(mapper2try)

lines2=lines2.map(lambda x:(x[1],x[2]))#只取jv
lines2 = lines2.collect()[250000:500000]
lines2= sc.parallelize(lines2)
print(lines2.collect())

[(0, 1), (0, 2), (0, 3), (1, 4), (1, 5), (1, 0), (2, 6), (2, 7), (2, 8)]


In [8]:
lines3 = lines1.join(lines2)#同j的v包一起
print(lines3.collect())

[(0, (10, 1)), (0, (10, 2)), (0, (10, 3)), (0, (0, 1)), (0, (0, 2)), (0, (0, 3)), (0, (40, 1)), (0, (40, 2)), (0, (40, 3)), (2, (20, 6)), (2, (20, 7)), (2, (20, 8)), (2, (0, 6)), (2, (0, 7)), (2, (0, 8)), (2, (50, 6)), (2, (50, 7)), (2, (50, 8)), (1, (0, 4)), (1, (0, 5)), (1, (0, 0)), (1, (30, 4)), (1, (30, 5)), (1, (30, 0)), (1, (0, 4)), (1, (0, 5)), (1, (0, 0))]


In [9]:
lines3 = lines3.values()
lines3 = lines3.map(lambda x:(x[0]*x[1]))#同j的v相乘，有n^3項
print(lines3.collect())

[10, 20, 30, 0, 0, 0, 40, 80, 120, 120, 140, 160, 0, 0, 0, 300, 350, 400, 0, 0, 0, 120, 150, 0, 0, 0, 0]


In [12]:
lines3 = sc.parallelize(lines3.collect(),500).glom()#分成500等分，每份的第n項相加起來是新矩陣的第1~250000項
print(lines3.collect())

[[10, 20, 30, 0, 0, 0, 40, 80, 120], [120, 140, 160, 0, 0, 0, 300, 350, 400], [0, 0, 0, 120, 150, 0, 0, 0, 0]]


In [13]:
path = '/Users/huangyushu/Desktop/Outputfile.txt'
data=open(path,'w+') 
tmp=0
for i in range(0,250000):
    for n in range(0,500):
         tmp+=lines3.collect()[n][i]
    print(str(i),str(tmp),file=data)#輸出形式為(i v) i:(0~249999)0->(0,0))2->0,1......249999->(499,499) v:value
    tmp=0
data.close()

130
160
190
120
150
0
340
430
520
